In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


In [3]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
image_dir = "/content/drive/MyDrive/images"
gt_dir = "/content/drive/MyDrive/GroundTruths"
result_dir = "/content/results"

os.makedirs(result_dir, exist_ok=True)


In [5]:
image_ids = [2018, 3063, 5096, 6046, 8068]

results = []


In [6]:
def prewitt_edge(img_gray):
    kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
    kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
    x = cv2.filter2D(img_gray, -1, kernelx)
    y = cv2.filter2D(img_gray, -1, kernely)
    prewitt = cv2.magnitude(np.float32(x), np.float32(y))
    return np.uint8(prewitt)


In [7]:
def evaluate_edges(pred, gt):

    pred = (pred > 0).astype(np.uint8)
    gt = (gt > 0).astype(np.uint8)

    pred_flat = pred.flatten()
    gt_flat = gt.flatten()

    precision = precision_score(gt_flat, pred_flat, zero_division=0)
    recall = recall_score(gt_flat, pred_flat, zero_division=0)
    f1 = f1_score(gt_flat, pred_flat, zero_division=0)
    accuracy = accuracy_score(gt_flat, pred_flat) * 100

    return precision, recall, f1, accuracy


In [9]:

for img_id in image_ids:

    print(f"\nProcessing image {img_id}...")
    img_path = os.path.join(image_dir, f"{img_id}.jpg")
    gt_path = os.path.join(gt_dir, f"{img_id}.png")

    img = cv2.imread(img_path)
    gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Edge Detectors

    sobel = np.uint8(cv2.magnitude(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3),
                                   cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)))
    prewitt = prewitt_edge(gray)
    laplacian = np.uint8(np.absolute(cv2.Laplacian(gray, cv2.CV_64F)))
    canny = cv2.Canny(gray, 100, 200)

    # Evaluation

    sobel_p, sobel_r, sobel_f1, sobel_acc = evaluate_edges(sobel, gt)
    prew_p, prew_r, prew_f1, prew_acc = evaluate_edges(prewitt, gt)
    lap_p, lap_r, lap_f1, lap_acc = evaluate_edges(laplacian, gt)
    canny_p, canny_r, canny_f1, canny_acc = evaluate_edges(canny, gt)

    results.append([img_id, sobel_f1, prew_f1, lap_f1, canny_f1, sobel_acc, prew_acc, lap_acc, canny_acc])

    # Display Comparison Figure

    fig, axs = plt.subplots(1, 6, figsize=(24, 6))

    axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[0].set_title(f"Original ({img_id})")

    axs[1].imshow(sobel, cmap='gray')
    axs[1].set_title("Sobel")

    axs[2].imshow(prewitt, cmap='gray')
    axs[2].set_title("Prewitt")

    axs[3].imshow(laplacian, cmap='gray')
    axs[3].set_title("Laplacian")

    axs[4].imshow(canny, cmap='gray')
    axs[4].set_title("Canny")

    axs[5].imshow(gt, cmap='gray')
    axs[5].set_title("Ground Truth")

    for ax in axs:
        ax.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(result_dir, f"{img_id}_comparison.png"))
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

In [17]:
df = pd.DataFrame(results, columns=[

    "Image ID", "Sobel F1", "Prewitt F1", "Laplacian F1", "Canny F1",
    "Sobel Acc (%)", "Prewitt Acc (%)", "Laplacian Acc (%)", "Canny Acc (%)"
])

print("\n Quantitative Results (F1 Scores + Accuracy %) \n")
df



 Quantitative Results (F1 Scores + Accuracy %) 



,Image ID,Sobel F1,Prewitt F1,Laplacian F1,Canny F1,Sobel Acc (%),Prewitt Acc (%),Laplacian Acc (%),Canny Acc (%)
0,2018,0.269066,0.256398,0.272288,0.255898,16.895616,36.583312,21.124863,80.553235
1,3063,0.251767,0.241339,0.251575,0.105375,15.100291,34.189545,21.607373,85.252686
2,5096,0.292474,0.283459,0.293655,0.232428,17.393670,35.396144,19.489511,78.371902
3,6046,0.332819,0.320409,0.333730,0.309607,27.192181,40.982895,28.167564,73.024786
4,8068,0.297173,0.284480,0.296349,0.134280,19.766711,37.254292,23.038711,81.276676
5,2018,0.269066,0.256398,0.272288,0.255898,16.895616,36.583312,21.124863,80.553235
6,3063,0.251767,0.241339,0.251575,0.105375,15.100291,34.189545,21.607373,85.252686
7,5096,0.292474,0.283459,0.293655,0.232428,17.393670,35.396144,19.489511,78.371902
8,6046,0.332819,0.320409,0.333730,0.309607,27.192181,40.982895,28.167564,73.024786
9,8068,0.297173,0.284480,0.296349,0.134280,19.766711,37.254292,23.038711,81.276676


In [ ]:
# save to CSV

df.to_csv(os.path.join(result_dir, "edge_detection_results.csv"), index=False)
